In [1]:
# %%capture
# %pip install torch
# %pip install torchvision==0.17
# %pip install torchsummary

In [2]:
import torch
torch.__version__

'2.2.0+cpu'

In [1]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms

from torchsummary import summary

In [2]:
# Define dictionary of arguments
args = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'batch_size': 64,
    'num_classes': 10,
    'num_epochs': 5,
    'lr': 0.01
}


transform = transforms.Compose([transforms.Resize((224,224) ),
                                transforms.Grayscale(num_output_channels=3), # Convert to 3-channel
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                                ])

# Define the batch size
mnist_batch_size = args['batch_size']

# Load the MNIST training dataset
mnist_trainset = torchvision.datasets.MNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transform
)

# Create a DataLoader for the MNIST training dataset
mnist_trainloader = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=mnist_batch_size,
    shuffle=True,
    num_workers=2
)

# Load the MNIST test dataset
mnist_testset = torchvision.datasets.MNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform
)

# Create a DataLoader for the MNIST test dataset
mnist_testloader = torch.utils.data.DataLoader(
    mnist_testset,
    batch_size=mnist_batch_size,
    shuffle=False,
    num_workers=2
)

In [3]:
# Define the transformation with ToTensor and Normalize for SVHN
svhn_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the batch size
svhn_batch_size = args['batch_size']

# Load the SVHN training dataset
svhn_trainset = torchvision.datasets.SVHN(
    root='./data/',
    split='train',
    download=True,
    transform=svhn_transform
)

# Create a DataLoader for the SVHN training dataset
svhn_trainloader = torch.utils.data.DataLoader(
    svhn_trainset,
    batch_size=svhn_batch_size,
    shuffle=True,
    num_workers=2
)

# Load the SVHN test dataset
svhn_testset = torchvision.datasets.SVHN(
    root='./data/',
    split='test',
    download=True,
    transform=svhn_transform
)

# Create a DataLoader for the SVHN test dataset
svhn_testloader = torch.utils.data.DataLoader(
    svhn_testset,
    batch_size=svhn_batch_size,
    shuffle=False,
    num_workers=2
)

# # Optionally, load the extra dataset if needed
# svhn_extraset = torchvision.datasets.SVHN(
#     root='./data/',
#     split='extra',
#     download=True,
#     transform=svhn_transform
# )

# # Create a DataLoader for the SVHN extra dataset
# svhn_extraloader = torch.utils.data.DataLoader(
#     svhn_extraset,
#     batch_size=svhn_batch_size,
#     shuffle=True,
#     num_workers=2
# )

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


### MMD calculation (MNIST vs SVHN)

In [6]:
# Define the Gaussian(RBF) kernel function
def gaussian_kernel(x, y, sigma=1.0):
    return torch.exp(-torch.sum((x - y) ** 2) / (2 * sigma ** 2))

# Compute MMD
def compute_mmd(x, y, kernel=gaussian_kernel, sigma=1.0):
    n = x.size(0)
    m = y.size(0)
    
    xx_kernel_sum = torch.sum(torch.tensor([kernel(xi, xj, sigma) for xi in x for xj in x])).item() / (n * n)
    yy_kernel_sum = torch.sum(torch.tensor([kernel(yi, yj, sigma) for yi in y for yj in y])).item() / (m * m)
    xy_kernel_sum = torch.sum(torch.tensor([kernel(xi, yj, sigma) for xi in x for yj in y])).item() / (n * m)
    
    return xx_kernel_sum + yy_kernel_sum - 2 * xy_kernel_sum

# Get samples from the loaders
mnist_samples, _ = next(iter(mnist_trainloader))
svhn_samples, _ = next(iter(svhn_trainloader))z

# Compute MMD
mmd_value = compute_mmd(mnist_samples, svhn_samples)
print(f'MMD value between MNIST and SVHN: {mmd_value}')

MMD value between MNIST and SVHN: 0.03125


In [ ]:
# MNIST samples
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, ax in enumerate(axes):
    ax.imshow(mnist_samples[i].permute(1, 2, 0).numpy() / 2 + 0.5)
    ax.set_title('MNIST')
    ax.axis('off')

# SVHN samples
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, ax in enumerate(axes):
    ax.imshow(svhn_samples[i].permute(1, 2, 0).numpy() / 2 + 0.5)
    ax.set_title('SVHN')
    ax.axis('off')

plt.show()

### CORAL implementation

In [4]:
class CORAL(nn.Module):
    def __init__(self):
        super(CORAL, self).__init__()

    def forward(self, source, target):
        d = source.size(1)

        # Compute covariance matrices
        source_c = self.covariance_matrix(source)
        target_c = self.covariance_matrix(target)

        # Compute the CORAL loss
        loss = self.coral_loss(source_c, target_c, d)
        return loss

    def covariance_matrix(self, x):
        n = x.size(0)
        x_centered = x - x.mean(0)
        cov = (x_centered.t() @ x_centered) / (n - 1)
        return cov

    def coral_loss(self, source_c, target_c, d):
        # Frobenius norm of the difference between the covariance matrices
        loss = torch.norm(source_c - target_c, p='fro') ** 2
        # Normalize by the dimensionality of the feature space
        loss = loss / (4 * d * d)
        return loss

# Define your neural network model
class SimpleNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Add more layers as needed
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 16 * 16, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Training loop
def train_model(model, source_loader, target_loader, criterion, coral_criterion, optimizer, device):
    model.train()

    for i, ((source_images, _), (target_images, _)) in enumerate(zip(source_loader, target_loader)):
        source_images = source_images.to(device)
        target_images = target_images.to(device)

        # Forward pass
        source_features = model.features(source_images)
        target_features = model.features(target_images)

        # Compute CORAL loss
        coral_loss = coral_criterion(source_features.view(source_features.size(0), -1),
                                     target_features.view(target_features.size(0), -1))

        # Compute classification loss (if you have labels)
        # class_loss = criterion(source_predictions, source_labels)

        # Combine losses
        loss = coral_loss  # + class_loss (if using classification loss)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Batch [{i + 1}], CORAL Loss: {coral_loss.item():.4f}')


# Instantiate the model, criterion, and optimizer
simpleNN = SimpleNN().to(args['device'])
coral_criterion = CORAL().to(args['device'])
optimizer = optim.Adam(simpleNN.parameters(), lr=0.001)

# Train the model
train_model(simpleNN, mnist_trainloader, svhn_trainloader, None, coral_criterion, optimizer, args['device'])


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2578054119424 bytes.